In [2]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

# Custom PeepholeLSTM layer
class PeepholeLSTM(Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        super(PeepholeLSTM, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                      initializer='glorot_uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units * 4),
                                                initializer='orthogonal',
                                                name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units * 4,),
                                    initializer='zeros',
                                    name='bias')
        self.cell_bias = self.add_weight(shape=(self.units * 3,),
                                         initializer='ones',
                                         name='cell_bias')
        self.built = True

    def call(self, inputs, states):
        h_tm1 = states[0]
        c_tm1 = states[1]
        z = K.dot(inputs, self.kernel)
        z += K.dot(h_tm1, self.recurrent_kernel)
        z += self.bias

        z0 = z[:, :self.units]  # Input gate
        z1 = z[:, self.units:2 * self.units]  # Forget gate
        z2 = z[:, 2 * self.units:3 * self.units]  # Cell gate
        z3 = z[:, 3 * self.units:]  # Output gate

        i = K.hard_sigmoid(z0 + self.cell_bias[:self.units] * c_tm1)  # Peephole connection for input gate
        f = K.hard_sigmoid(z1 + self.cell_bias[self.units:2 * self.units] * c_tm1)  # Peephole connection for forget gate
        c = f * c_tm1 + i * K.tanh(z2)
        o = K.hard_sigmoid(z3 + self.cell_bias[2 * self.units:] * c)  # Peephole connection for output gate

        h = o * K.tanh(c)
        return h, [h, c]

# Set the path to your dataset
train_data_dir = r"D:\P_126\data_idumfc\preprocessed_train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the Peephole LSTM model using the custom PeepholeLSTM layer
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))  # Standard LSTM
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=30)

# Save the trained model
model.save('lung_detection_standard_lstm.h5')


Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 746s 1s/step - accuracy: 0.4921 - loss: 1.2587
Epoch 2/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 341s 677ms/step - accuracy: 0.7880 - loss: 0.5768
Epoch 3/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 323s 644ms/step - accuracy: 0.8740 - loss: 0.3470
Epoch 4/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 328s 654ms/step - accuracy: 0.9328 - loss: 0.1935
Epoch 5/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 324s 646ms/step - accuracy: 0.9641 - loss: 0.0967
Epoch 6/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 322s 641ms/step - accuracy: 0.9726 - loss: 0.0764
Epoch 7/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 328s 653ms/step - accuracy: 0.9882 - loss: 0.0410
Epoch 8/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 329s 655ms/step - accuracy: 0.9809 - loss: 0.0571
Epoch 9/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 325s 647ms/step - accuracy: 0.9903 - loss: 0.0294
Epoch 10/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 329s 656ms/step - accuracy: 0.9896 - loss: 0.0313
Epoch 11/30
502/502 ━━━━━━━━━━━━━━━━━━━━ 324s 646ms/step - accuracy: 0.9886 - loss: 0.0331
Epoch 12/30

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# Load the trained model
model = tf.keras.models.load_model('lung_detection_standard_lstm.h5')

# Prepare your test dataset (similar to how you prepared the training data)
test_data_dir = r"D:\P_126\data_idumfc\preprocessed_test"
img_width, img_height = 150, 150
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_true = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_true.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_true = np.array(y_true, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_true)

# Make predictions on the test data
y_pred_probs = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compute precision, recall, and confusion matrix
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Found 4110 images belonging to 6 classes.
129/129 ━━━━━━━━━━━━━━━━━━━━ 15s 106ms/step - accuracy: 0.7420 - loss: 2.7091
129/129 ━━━━━━━━━━━━━━━━━━━━ 14s 109ms/step
Test Loss: 2.4907796382904053
Test Accuracy: 0.7476885914802551
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.80      0.71      0.75       685
       COVID       0.70      0.64      0.67       685
      NORMAL       0.73      0.83      0.78       685
   PNEUMONIA       0.96      0.80      0.87       685
PNEUMOTHORAX       0.55      0.71      0.62       685
TUBERCULOSIS       0.84      0.79      0.82       685

    accuracy                           0.75      4110
   macro avg       0.77      0.75      0.75      4110
weighted avg       0.77      0.75      0.75      4110

Confusion Matrix:
[[483  64   3   2 121  12]
 [ 45 439  35   4 108  54]
 [ 16  13 571  11  71   3]
 [  5  12  94 549  18   7]
 [ 32  66  71   5 487  24]
 [ 20  35   3   1  82 544]]


In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

# Custom PeepholeLSTM layer
class PeepholeLSTM(Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        super(PeepholeLSTM, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                      initializer='glorot_uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units * 4),
                                                initializer='orthogonal',
                                                name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units * 4,),
                                    initializer='zeros',
                                    name='bias')
        self.cell_bias = self.add_weight(shape=(self.units * 3,),
                                         initializer='ones',
                                         name='cell_bias')
        self.built = True

    def call(self, inputs, states):
        h_tm1 = states[0]
        c_tm1 = states[1]
        z = K.dot(inputs, self.kernel)
        z += K.dot(h_tm1, self.recurrent_kernel)
        z += self.bias

        z0 = z[:, :self.units]  # Input gate
        z1 = z[:, self.units:2 * self.units]  # Forget gate
        z2 = z[:, 2 * self.units:3 * self.units]  # Cell gate
        z3 = z[:, 3 * self.units:]  # Output gate

        i = K.hard_sigmoid(z0 + self.cell_bias[:self.units] * c_tm1)  # Peephole connection for input gate
        f = K.hard_sigmoid(z1 + self.cell_bias[self.units:2 * self.units] * c_tm1)  # Peephole connection for forget gate
        c = f * c_tm1 + i * K.tanh(z2)
        o = K.hard_sigmoid(z3 + self.cell_bias[2 * self.units:] * c)  # Peephole connection for output gate

        h = o * K.tanh(c)
        return h, [h, c]

# Set the path to your dataset
train_data_dir = r"D:\P_126\data_idumfc\preprocessed_train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the Peephole LSTM model using the custom PeepholeLSTM layer
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))  # Standard LSTM
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=20)

# Save the trained model
model.save('lung_detection_standard_lstm20.h5')


Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 702s 1s/step - accuracy: 0.4559 - loss: 1.3623
Epoch 2/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 651s 1s/step - accuracy: 0.7523 - loss: 0.6619
Epoch 3/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 648s 1s/step - accuracy: 0.8316 - loss: 0.4527
Epoch 4/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 644s 1s/step - accuracy: 0.8902 - loss: 0.2878
Epoch 5/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 628s 1s/step - accuracy: 0.9365 - loss: 0.1735
Epoch 6/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 646s 1s/step - accuracy: 0.9560 - loss: 0.1194
Epoch 7/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 687s 1s/step - accuracy: 0.9721 - loss: 0.0857
Epoch 8/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 649s 1s/step - accuracy: 0.9811 - loss: 0.0567
Epoch 9/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 685s 1s/step - accuracy: 0.9808 - loss: 0.0550
Epoch 10/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 654s 1s/step - accuracy: 0.9876 - loss: 0.0395
Epoch 11/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 504s 1s/step - accuracy: 0.9819 - loss: 0.0540
Epoch 12/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# Load the trained model
model = tf.keras.models.load_model('lung_detection_standard_lstm20.h5')

# Prepare your test dataset (similar to how you prepared the training data)
test_data_dir = r"D:\P_126\data_idumfc\preprocessed_test"
img_width, img_height = 150, 150
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_true = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_true.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_true = np.array(y_true, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_true)

# Make predictions on the test data
y_pred_probs = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compute precision, recall, and confusion matrix
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Found 4110 images belonging to 6 classes.
129/129 ━━━━━━━━━━━━━━━━━━━━ 14s 101ms/step - accuracy: 0.7549 - loss: 1.8669
129/129 ━━━━━━━━━━━━━━━━━━━━ 15s 116ms/step
Test Loss: 1.8091752529144287
Test Accuracy: 0.7535279989242554
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.79      0.74      0.76       685
       COVID       0.68      0.64      0.66       685
      NORMAL       0.74      0.81      0.77       685
   PNEUMONIA       0.90      0.85      0.88       685
PNEUMOTHORAX       0.61      0.64      0.62       685
TUBERCULOSIS       0.81      0.84      0.83       685

    accuracy                           0.75      4110
   macro avg       0.76      0.75      0.75      4110
weighted avg       0.76      0.75      0.75      4110

Confusion Matrix:
[[508  68   4   8  69  28]
 [ 57 439  29   9  90  61]
 [ 13  14 554  33  68   3]
 [  7  18  58 585  16   1]
 [ 41  63  93  12 435  41]
 [ 18  39  15   4  33 576]]


In [3]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

# Custom PeepholeLSTM layer
class PeepholeLSTM(Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        super(PeepholeLSTM, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                      initializer='glorot_uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units * 4),
                                                initializer='orthogonal',
                                                name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units * 4,),
                                    initializer='zeros',
                                    name='bias')
        self.cell_bias = self.add_weight(shape=(self.units * 3,),
                                         initializer='ones',
                                         name='cell_bias')
        self.built = True

    def call(self, inputs, states):
        h_tm1 = states[0]
        c_tm1 = states[1]
        z = K.dot(inputs, self.kernel)
        z += K.dot(h_tm1, self.recurrent_kernel)
        z += self.bias

        z0 = z[:, :self.units]  # Input gate
        z1 = z[:, self.units:2 * self.units]  # Forget gate
        z2 = z[:, 2 * self.units:3 * self.units]  # Cell gate
        z3 = z[:, 3 * self.units:]  # Output gate

        i = K.hard_sigmoid(z0 + self.cell_bias[:self.units] * c_tm1)  # Peephole connection for input gate
        f = K.hard_sigmoid(z1 + self.cell_bias[self.units:2 * self.units] * c_tm1)  # Peephole connection for forget gate
        c = f * c_tm1 + i * K.tanh(z2)
        o = K.hard_sigmoid(z3 + self.cell_bias[2 * self.units:] * c)  # Peephole connection for output gate

        h = o * K.tanh(c)
        return h, [h, c]

# Set the path to your dataset
train_data_dir = r"D:\P_126\data_idumfc\preprocessed_train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the Peephole LSTM model using the custom PeepholeLSTM layer
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))  # Standard LSTM
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=10)




Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 331s 641ms/step - accuracy: 0.4735 - loss: 1.3091
Epoch 2/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 317s 632ms/step - accuracy: 0.7776 - loss: 0.6017
Epoch 3/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 316s 629ms/step - accuracy: 0.8684 - loss: 0.3704
Epoch 4/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 318s 633ms/step - accuracy: 0.9255 - loss: 0.2038
Epoch 5/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 319s 634ms/step - accuracy: 0.9636 - loss: 0.1108
Epoch 6/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 354s 705ms/step - accuracy: 0.9773 - loss: 0.0715
Epoch 7/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 349s 696ms/step - accuracy: 0.9820 - loss: 0.0517
Epoch 8/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 337s 671ms/step - accuracy: 0.9832 - loss: 0.0497
Epoch 9/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 350s 698ms/step - accuracy: 0.9912 - loss: 0.0301
Epoch 10/10
502/502 ━━━━━━━━━━━━━━━━━━━━ 636s 1s/step - accuracy: 0.9949 - loss: 0.0179


RuntimeError: Can't decrement id ref count (unable to extend file properly)

In [4]:
# Save the trained model
model.save('lung_detection_standard_lstm10.h5')

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# Load the trained model
model = tf.keras.models.load_model('lung_detection_standard_lstm10.h5')

# Prepare your test dataset (similar to how you prepared the training data)
test_data_dir = r"D:\P_126\data_idumfc\preprocessed_test"
img_width, img_height = 150, 150
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_true = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_true.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_true = np.array(y_true, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_true)

# Make predictions on the test data
y_pred_probs = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compute precision, recall, and confusion matrix
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Found 4110 images belonging to 6 classes.
129/129 ━━━━━━━━━━━━━━━━━━━━ 47s 259ms/step - accuracy: 0.7576 - loss: 1.6802
129/129 ━━━━━━━━━━━━━━━━━━━━ 68s 272ms/step
Test Loss: 1.61447012424469
Test Accuracy: 0.7549878358840942
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.77      0.75      0.76       685
       COVID       0.73      0.68      0.70       685
      NORMAL       0.75      0.77      0.76       685
   PNEUMONIA       0.93      0.85      0.89       685
PNEUMOTHORAX       0.60      0.55      0.57       685
TUBERCULOSIS       0.75      0.92      0.83       685

    accuracy                           0.75      4110
   macro avg       0.75      0.75      0.75      4110
weighted avg       0.75      0.75      0.75      4110

Confusion Matrix:
[[513  55   4   8  72  33]
 [ 44 468  26   7  61  79]
 [ 14  17 529  25  79  21]
 [  5  10  55 585  20  10]
 [ 82  77  81   5 375  65]
 [ 12  18   8   0  14 633]]
